Mount Google Drive (optional)

In [353]:
from google.colab import drive
drive.mount('/content/drive')

import os
# os.chdir("/content/drive/MyDrive/....")  # file path
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


# **HW2 : Decision Tree and Random Forest**
In *assignment 2*, you need to finish :

1. Basic Part : Implement a **Decision Tree** model and predict whether the patients in the validation set have diabetes
> * Step 1 : Load the input data
> * Step 2 : Calculate the Entropy and Information Gain
> * Step 3 : Find the Best Split
> * Step 4 : Split into 2 branches
> * Step 5 : Build decision tree
> * Step 6 : Save the answers from step2 to step5
> * Step 7 : Split data into training set and validation set
> * Step 8 : Train a decision tree model with training set
> * Step 9 : Predict the cases in the *validation set* by using the model trained in *Step8*
> * Step 10 : Calculate the f1-score of your predictions in *Step9*
> * Step 11 : Write the Output File

2. Advanced Part : Build a **Random Forest** model to make predictions
> * Step 1 : Load the input data
> * Step 2 : Load the test data
> * Step 3 : Build a random forest
> * Step 4 : Predict the cases in the test data by using the model trained in *Step3*
> * Step 5 : Save the predictions(from *Step 4*) in a csv file



# **Basic Part** (60%)
In this part, your need to implement a Decision Tree model by completing the following given functions.

Also, you need to run these functions with the given input variables and save the output in a csv file **hw2_basic.csv**

## Import Packages


> Note : You **cannot** import any other packages in both basic part and advanced part






In [354]:
import numpy as np
import pandas as pd
import math
import random
from numpy import sqrt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

## Step1: Load the input data
First, load the input file **hw2_input_basic.csv**

In [355]:
input_data = pd.read_csv('hw2_input_basic.csv')
input_data

,age,bmi,gender,height,weight,glucose_apache,heart_rate_apache,resprate_apache,sodium_apache,diabetes_mellitus
0,70.0,25.984659,1,172.7,77.50,116.0,101.0,49.0,137.0,0
1,30.0,31.310368,1,170.2,90.70,71.0,39.0,33.0,144.0,0
2,54.0,24.388824,1,177.8,77.10,120.0,120.0,31.0,141.0,0
3,65.0,34.141074,0,170.2,98.90,73.0,48.0,36.0,140.0,1
4,49.0,22.564743,1,172.7,67.30,207.0,119.0,6.0,144.0,0
5,62.0,29.424010,0,154.9,70.60,113.0,60.0,32.0,137.0,0
6,85.0,27.673574,1,154.9,66.40,102.0,49.0,36.0,142.0,0
7,65.0,22.269432,1,177.8,70.40,333.0,59.0,6.0,145.0,1
8,85.0,35.879362,0,165.1,97.80,124.0,92.0,30.0,136.0,0
9,81.0,20.859375,0,160.0,53.40,136.0,118.0,52.0,138.0,0


## Global attributes
Define the global attributes
> Note : You **cannot** modify the values of these attributes we given in the basic part

In [356]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = input_data.shape[1] - 1

> You can add your own global attributes here

## Step2 : Calculate the Entropy and Information Gain 
Calculate the information gain and entropy values before separate data into left subtree and right subtree

In [357]:
def entropy(data):
  """
  This function measures the amount of uncertainty in a probability distribution
  args: 
  * data(type: DataFrame): the data you're calculating for the entropy
  return:
  * entropy_value(type: float): the data's entropy
  """
  dia=data['diabetes_mellitus']
  n = np.shape(data)[0]
  a=dia.value_counts()/n
  entropy_value=np.sum(-a*np.log2(a+1e-9))
  return entropy_value

# [Note] You have to save the value of "ans_entropy" into the output file
ans_entropy = entropy(input_data)
print("ans_entropy = ", ans_entropy)

ans_entropy =  0.9871377714867962


In [358]:
def information_gain(data, mask):
  """
  This function will calculate the information gain
  args:
  * data(type: DataFrame): the data you're calculating for the information gain
  * mask(type: Series): partition information(left/right) of current input data, 
    - boolean 1(True) represents split to left subtree
    - boolean 0(False) represents split to right subtree
  return:
  * ig(type: float): the information gain you can obtain by classify data with this given mask
  """
  
  a=sum(mask)
  b=mask.shape[0]-a
  
  truenum=a/(a+b)
  falsenum=b/(a+b)
  # if sum(mask)==0 or mask.shape[0]-sum(mask)==0:
  #   return 0
  if truenum==0 or falsenum==0:
    return 0
  

  if len(data)!=len(mask):
     print("fuck u")
     print(mask)
     print(data)
  true_data=data[mask]#true data
  false_data=data[-mask]#false data
  
  
  ig=entropy(data)-truenum*entropy(true_data)-falsenum*entropy(false_data)
  

  return ig

# [Note] You have to save the value of "ans_informationGain" into your output file
temp1 = np.zeros((int(input_data.shape[0]/4), 1), dtype=bool)
temp2 = np.ones(((input_data.shape[0]-int(input_data.shape[0]/4), 1)), dtype=bool)
temp_mask = np.concatenate((temp1, temp2))
df_mask = pd.DataFrame(temp_mask, columns=['mask'])

ans_informationGain = information_gain(input_data, df_mask['mask'])
print("ans_informationGain = ", ans_informationGain)

ans_informationGain =  0.08345988684807129


## Step3 : Find the Best Split
Find the best split combination, **feature** and **threshold**, by calculating the information gain


In [359]:
def maxfind(x,d):
  ig=[]
  split_value=[]
  options = x.sort_values().unique()
  for val in options:
    mask = (x <= val)
    val_ig = information_gain(d,mask)
    # Append results
    ig.append(val_ig)
    split_value.append(val) #分開值
  best_ig = max(ig)
  best_ig_index = ig.index(best_ig)
  best_split = split_value[best_ig_index]
  return(best_ig,best_split)
  
def find_best_split(data):
  """
  This function will find the best split combination of data
  args:
  * data(type: DataFrame): the input data
  return
  * best_ig(type: float): the best information gain you obtain
  * best_threshold(type: float): the value that splits data into 2 branches
  * best_feature(type: string): the feature that splits data into 2 branches
  """
  result=data.drop('diabetes_mellitus', axis= 1).apply(maxfind,d=data)
  ff=result
  ff.index=range(0,2)
  #print(ff)
 
  best_ig=ff.max(axis=1)[0]
  best_feature=ff.idxmax(axis=1)[0]
  best_threshold=result[best_feature][1]
  # print(best_threshold)
  return best_ig, best_threshold, best_feature


# [Note] You have to save the value of "ans_ig", "ans_value", and "ans_name" into the output file
ans_ig, ans_value, ans_name = find_best_split(input_data)
print("ans_ig = ", ans_ig)
print("ans_value = ", ans_value)
print("ans_name = ", ans_name)

ans_ig =  0.3522950512446044
ans_value =  207.0
ans_name =  glucose_apache


## Step4 : Split into 2 branches
Using the best split combination you find in function *find_best_split()* to split data into Left Subtree and Right Subtree 

In [360]:
def make_partition(data, feature, threshold):
  """
  This function will split the data into 2 branches
  args:
  * data(type: DataFrame): the input data
  * feature(type: string): the attribute(column name)
  * threshold(type: float): the threshold for splitting the data
  return:
  * left(type: DataFrame): the divided data that matches(less than or equal to) the assigned feature's threshold
  * right(type: DataFrame): the divided data that doesn't match the assigned feature's threshold
  """
  left=data[data[feature]<threshold]
  right=data[(data[feature]<threshold)==False]
  



  
  return left, right


# [Note] You have to save the value of "ans_left" into the output file
left, right = make_partition(input_data, 'age', 61.0)
ans_left = left.shape[0]
print("ans_left = ", ans_left)


ans_left =  9


## Step5 : Build Decision Tree
Use the above functions to implement the decision tree

Instructions: 
1.  If current depth < max_depth and the remaining number of samples > min_samples_split: continue to classify those samples
2.  Use function *find_best_split()* to find the best split combination
3.  If the obtained information gain is **greater than 0**: can build a deeper decision tree (add depth)
4. Use function *make_partition()* to split the data into two parts
5. Save the features and corresponding thresholds (starting from the root) used by the decision tree into *ans_features[]* and *ans_thresholds[]* respectively




In [361]:
def build_tree(data, max_depth, min_samples_split, depth):
  """
  This function will build the decision tree
  args:
  * data(type: DataFrame): the data you want to apply to the decision tree
  * max_depth: the maximum depth of a decision tree
  * min_samples_split: the minimum number of instances required to do partition
  * depth: the height of the current decision tree
  return:
  * subtree: the decision tree structure including root, branch, and leaf (with the attributes and thresholds)
  """
  # check the condition of current depth and the remaining number of samples
  if depth < max_depth :
    # call find_best_split() to find the best combination
    split_ig, split_value, split_name=find_best_split(data)
    # check the value of information gain is greater than 0 or not 
    if split_ig > 0 :
      # update the depth
      depth+=1
      # call make_partition() to split the data into two parts
      l,r=make_partition(data,split_name,split_value)
      # If there is no data split to the left tree OR no data split to the left tree
      if l.shape[0]<1 or r.shape[0]<1 :
        # return the label of the majority
        label = data['diabetes_mellitus'].value_counts().idxmax()
        return label
      else:
        question = "{} {} {}".format(split_name, "<=", split_value)
        subtree = {question: []}

        # call function build_tree() to recursively build the left subtree and right subtree
        left_subtree = build_tree(l, max_depth, min_samples_split, depth)
        right_subtree = build_tree(r, max_depth, min_samples_split, depth)
        if left_subtree == right_subtree:
          subtree = left_subtree
        else:
          subtree[question].append(left_subtree)
          subtree[question].append(right_subtree)
    else:
      # return the label of the majority
      label =data['diabetes_mellitus'].value_counts().idxmax()
      return label
  else:
    # return the label of the majority
    label = data['diabetes_mellitus'].value_counts().idxmax()
    return label

  return subtree

An example of the output from *build_tree()* 
```
{'bmi <= 33.5': [1, {'age <= 68.5': [0, 1]}]}
```
Therefore, 
```
ans_features = ['bmi', 'age']
ans_thresholds = [33.5, 68.5]
```



In [362]:
ans_features = []
ans_thresholds = []

decisionTree = build_tree(input_data, max_depth, min_samples_split, depth)
print(decisionTree)
treestr=str(decisionTree)
string_to_push=""
thres_to_push=""
for i in treestr:
  if i.isalpha() or i=="_":
    string_to_push+=i
  elif i==" ":
    if string_to_push!="":
      ans_features.append(string_to_push)
      string_to_push=""
  elif i.isdigit() or i==".":
    thres_to_push+=i
  elif i=='\'':
    if thres_to_push!="":
      ans_thresholds.append(thres_to_push)
      thres_to_push=""
  else:
    thres_to_push=""
    string_to_push=""
# print(ans_features)
# print(ans_thresholds)
#list(decisionTree.keys())[0].split()

{'glucose_apache <= 207.0': [{'heart_rate_apache <= 133.0': [0, 1]}, 1]}


In [363]:
# [Note] You have to save the features in the "decisionTree" structure (from root to branch and leaf) into the output file
ans_features


['glucose_apache', 'heart_rate_apache']

In [364]:
# [Note] You have to save the corresponding thresholds for the features in the "ans_features" list into the output file
ans_thresholds

['207.0', '133.0']

## Step6 : Save answers

In [365]:
basic = []
basic.append(ans_entropy)
basic.append(ans_informationGain)
basic.append(ans_ig)
basic.append(ans_value)
basic.append(ans_name)
basic.append(ans_left)
for i in range(len(ans_features)):
  basic.append(ans_features[i])
for m in range(len(ans_thresholds)):
  basic.append(ans_thresholds[m])

## Step7 : Split data
Split data into training set and validation set
> Note: We have split the data into training set and validation. You **cannot** change the distribution of the data.

In [366]:
num_train = 20
num_validation = 10

training_data = input_data.iloc[:num_train]
validation_data = input_data.iloc[-num_validation:]

y_train = training_data[["diabetes_mellitus"]] #前20diabetes
x_train = training_data.drop(['diabetes_mellitus'], axis=1) #前20參數

y_validation = validation_data[["diabetes_mellitus"]]
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)

y_validation = y_validation.values.flatten()


print(input_data.shape)
print(training_data.shape)
print(validation_data.shape)

(30, 10)
(20, 10)
(10, 10)


## Step8 to Step10 : Make predictions with a decision tree

Define the attributions of the decision tree
> You **cannot** modify the values of these attributes in this part

In [367]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = x_train.shape[1]

We have finished the function '*classify_data()*' below, however, you can modify this function if you prefer completing it on your own way.

In [368]:
def classify_data(instance, tree):
  """
  This function will predict/classify the input instance
  args:
  * instance: a instance(case) to be predicted
  return:
  * answer: the prediction result (the classification result)
  """
  equation = list(tree.keys())[0] 
  
  if equation.split()[1] == '<=':
    temp_feature = equation.split()[0]
    temp_threshold = equation.split()[2]
    if instance[temp_feature] > float(temp_threshold):
      answer = tree[equation][1]
    else:
      answer = tree[equation][0]
  else:
    if instance[equation.split()[0]] in (equation.split()[2]):
      answer = tree[equation][0]
    else:
      answer = tree[equation][1]

  if not isinstance(answer, dict):
    return answer
  else:
    return classify_data(instance, answer)


def make_prediction(tree, data):
  """
  This function will use your pre-trained decision tree to predict the labels of all instances in data
  args:
  * tree: the decision tree
  * data: the data to predict
  return:
  * y_prediction: the predictions
  """
  
  # [Note] You can call the function classify_data() to predict the label of each instance
  #print(data)
  y_prediction=[]
  num_obs = len(data)
 
  for i in range(num_obs):
    obs_pred = classify_data(data.iloc[i,:], tree)
    y_prediction.append(obs_pred)
  return y_prediction


def calculate_score(y_true, y_pred):
  """
  This function will calculate the f1-score of the predictions
  args:
  * y_true: the ground truth
  * y_pred: the predictions
  return:
  * score: the f1-score
  """
  score=f1_score(y_true, y_pred) 
  return score

In [369]:
decision_tree = build_tree(training_data, max_depth, min_samples_split, depth)

y_pred = make_prediction(decision_tree, x_validation)

# [Note] You have to save the value of "ans_f1score" the your output file
ans_f1score = calculate_score(y_validation, y_pred)
print("ans_f1score = ", ans_f1score)

ans_f1score =  0.5


## Step11 : Write the Output File
Save all of your answers in a csv file, named as **hw2_basic.csv**

In [370]:
ans_path = 'hw2_basic.csv'

# [Note] You have to save the value of "ans_f1score" into the output file
basic.append(ans_f1score)
print(basic)

pd.DataFrame(basic).to_csv(ans_path, header = None, index = None)

[0.9871377714867962, 0.08345988684807129, 0.3522950512446044, 207.0, 'glucose_apache', 9, 'glucose_apache', 'heart_rate_apache', '207.0', '133.0', 0.5]


# **Advanced Part** (35%)

## Step1: Load the input data
First, load the input file **hw2_input_advanced.csv**

In [371]:
advanced_data = pd.read_csv('hw2_input_advanced.csv')

You can split *advanced_data* into training set and validaiton set

In [384]:
num_train = int(8379*2/3)
num_validation = int(8379/3)

training_data = advanced_data.iloc[:num_train]
validation_data = advanced_data.iloc[-num_validation:]

validy = validation_data[["diabetes_mellitus"]]
validx = validation_data.drop(['diabetes_mellitus'], axis=1)



## Step2 : Load the test data
Load the input file **hw2_input_test.csv** to make predictions with the pre-trained random forest model

In [373]:
x_test = pd.read_csv('hw2_input_test.csv')
x_test

,age,bmi,gender,height,weight,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,...,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,apache_4a_hospital_death_prob,apache_4a_icu_death_prob
0,62,32.866392,1,177.80,103.9,1,31.0,10.30,4,6,...,36.4,0,157,26,134,36.1,0,4.56,0.06,0.03
1,82,23.582766,0,157.50,58.5,0,26.0,0.54,3,4,...,32.8,0,42,25,142,36.1,0,6.00,0.14,0.06
2,61,31.684520,1,172.70,94.5,0,16.0,1.11,4,6,...,35.3,0,129,6,131,36.8,0,8.59,0.05,0.03
3,58,45.156250,0,160.00,115.6,0,19.0,0.70,1,4,...,30.1,1,131,23,138,34.9,1,16.03,0.33,0.22
4,74,25.817016,1,172.70,77.0,0,25.0,0.93,4,6,...,34.5,0,55,12,135,36.3,0,45.80,0.12,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,73,17.943584,0,157.48,44.5,0,12.0,0.30,4,6,...,33.8,0,129,9,144,36.9,0,7.70,0.02,0.01
836,79,29.049732,1,167.60,81.6,0,48.0,2.19,4,6,...,42.7,0,163,9,139,36.4,0,10.77,0.06,0.03
837,85,24.627827,0,152.40,57.2,0,11.0,0.48,3,5,...,29.5,0,67,9,139,36.6,0,7.35,0.16,0.05
838,68,32.510940,1,193.00,121.1,0,14.0,0.64,4,6,...,37.5,0,61,10,140,36.9,1,16.02,0.00,0.00


## Step3 : Build a Random Forest

Define the attributions of the random forest
> * You **can** modify the values of these attributes in advanced part
> * Each tree can have different attribute values
> * There must be **at least** 3 decision trees in the random forest model
> * Must use function *build_tree()* to build a random forest model
> * These are the parameters you can adjust : 


    ```
    max_depth = 
    depth = 0
    min_samples_split = 
    
    # total number of trees in a random forest
    n_trees = 

    # number of features to train a decision tree
    n_features = 

    # the ratio to select the number of instances
    sample_size = 
    n_samples = int(training_data.shape[0] * sample_size)
    ```




In [374]:
# Define the attributes

max_depth = 7
depth = 0
min_samples_split = 11
    
# total number of trees in a random forest
n_trees = 15

# number of features to train a decision tree
n_features = 11

# the ratio to select the number of instances
sample_size = 0.25
n_samples = int(training_data.shape[0] * sample_size)


In [375]:
# input_data


In [376]:


def build_forest(data, n_trees, n_features, n_samples):
  """
  This function will build a random forest.
  args:
  * data: all data that can be used to train a random forest
  * n_trees: total number of tree
  * n_features: number of features
  * n_samples: number of instances
  return:
  * forest: a random forest with 'n_trees' of decision tree
  """
  drop_num=24-n_features #13
  all_features=[]
  for i in data:
    all_features.append(i)
  all_features.pop()
  
  forest=[]
  for i in range(0, n_trees):
    drop_idxs=random.randrange(0, len(all_features), drop_num)
    drop_list=[]
    
    for idx in range(drop_num):
      drop_list.append(all_features[idx])

    # print(drop_list)
    tmp=data.drop(drop_list,axis=1)
    # print(drop_list)
    # print(tmp)
    # bootstrap training dataset
    #
    sample=data.sample(n_samples)
    tree=build_tree(sample, max_depth, 2,0)
    #k_indices = np.random.choice(len(data), 10)
    print(tree)
    forest.append(tree)
  # must reuse function build_tree()
  #tree = build_tree(.....)

  return forest

In [377]:
forest = build_forest(advanced_data, n_trees, n_features, n_samples)

{'glucose_apache <= 162.0': [{'age <= 50.0': [{'bmi <= 31.65185643': [0, {'glucose_apache <= 73.0': [1, {'resprate_apache <= 13.0': [{'bun_apache <= 27.0': [{'glucose_apache <= 141.0': [0, 1]}, 1]}, 0]}]}]}, {'weight <= 99.8': [{'resprate_apache <= 13.0': [{'bun_apache <= 43.0': [0, {'age <= 81.0': [1, 0]}]}, {'glucose_apache <= 87.0': [{'apache_4a_hospital_death_prob <= 0.01': [1, 0]}, 0]}]}, {'hematocrit_apache <= 34.4': [{'gcs_verbal_apache <= 2.0': [1, {'bmi <= 38.20018365': [{'resprate_apache <= 30.0': [1, 0]}, 1]}]}, {'age <= 58.0': [0, {'glucose_apache <= 87.0': [{'sodium_apache <= 132.0': [0, 1]}, 0]}]}]}]}]}, {'glucose_apache <= 213.0': [{'resprate_apache <= 12.0': [{'map_apache <= 121.0': [{'temp_apache <= 34.4': [0, 1]}, 1]}, {'bmi <= 22.88729234': [{'bun_apache <= 60.0': [0, 1]}, {'age <= 35.0': [0, 1]}]}]}, {'wbc_apache <= 24.2': [{'apache_4a_icu_death_prob <= 0.13': [1, {'resprate_apache <= 38.0': [1, {'wbc_apache <= 5.1': [1, 0]}]}]}, {'glucose_apache <= 257.0': [{'map_a

## Step4 : Make predictions with the random forest
> Note: Please print the f1-score of the predictions of each decision tree

In [378]:
def make_prediction_forest(forest, data):
  """
  This function will use the pre-trained random forest to make the predictions
  args:
  * forest: the random forest
  * data: the data used to predict
  return:
  * y_prediction: the predicted results
  """
  predict_num=len(data)
  
  y_prediction=[]
  tmp=[0]*predict_num
  for tre in forest:
    a= make_prediction(tre, data)
    tmp=np.sum([tmp,a],axis=0).tolist()

  tmp=np.around(np.array(tmp)/n_trees)

  y_prediction=tmp
  

  return y_prediction

In [385]:
x_test=pd.read_csv('hw2_input_test.csv')

y_pred_test = make_prediction_forest(forest, x_test)
print(y_pred_test)



# score=make_prediction_forest(forest, validx)
# sscore=calculate_score(validy,score)
# print(sscore)

[0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1.
 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1.

## Step5 : Write the Output File
Save your predictions from the **random forest** in a csv file, named as **hw2_advanced.csv**

In [386]:
advanced = []
for i in range(len(y_pred_test)):
  advanced.append(y_pred_test[i])

In [387]:
advanced_path = 'hw2_advanced.csv'
pd.DataFrame(advanced).to_csv(advanced_path, header = None, index = None)